<a href="https://colab.research.google.com/github/raghunadh-eva/eva6/blob/master/EVA6_S4_ASSIGNMENT/PART2/EVA4_S4_ASSIGNMENT_PART2_AFTER6AM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        #28x28x32
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        #28x28x32
        self.pool1 = nn.MaxPool2d(2, 2)
        #14x14x32
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        #14x14x32
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        #14x14x32
        self.pool2 = nn.MaxPool2d(2, 2)
        #7x7x32
        self.conv5 = nn.Conv2d(32, 32, 3)
        #5x5x32
        self.conv6 = nn.Conv2d(32, 32, 3)
        #3x3x32
        self.conv7 = nn.Conv2d(32, 10, 3)
        #1x1x32

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        #print(x.shape)
        #print(x)
        x = x.view(-1, 10)
        #print(x.shape)
        #print(x)
        return F.log_softmax(x)

In [3]:
class NewNetwork(nn.Module):
    def __init__(self):
        super(NewNetwork, self).__init__()
        
        #Using the concepts of Block
        #Block1 - Hit receptive field of 5x5
        self.conv1 = nn.Sequential(
            #28x28x1 | 3x3x1x32 -> 28x28x32
            #28x28x32 | 3x3x32 -> 28x28x32
            #28x28x32 | 26x26x32
            #batch normalization --> amplify the features
            #maxpool2sd --> 12x12x32
            nn.Conv2d(1, 16, 3, padding =1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.05),
            nn.Conv2d(16, 16, 3, padding =1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.05),
            nn.Conv2d(16, 16, 3, padding =1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.05),
            nn.MaxPool2d(2, 2)
            ##Dont use dropoout in first pass
            #nn.Dropout()
        )
        
        self.conv2 = nn.Sequential(
            #14x14x32 | 3x3x1x32 -> 14x14x32
            #batch normalization --> amplify the features
            #maxpool2sd --> 12x12x32
            nn.Conv2d(16, 16, 3, padding =1),
            nn.ReLU(),
            nn.Dropout2d(0.05),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding =1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.05),
            nn.Conv2d(16, 16, 3, padding =1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2)
            #nn.Dropout2d(0.05)
        )
        #7x7x32 --> #7x7x32
        #Global average pooling
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding =1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding =1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding =1),
            nn.AvgPool2d(7)
        )
        #7x7x10
        self.fc = nn.Sequential(
            nn.Linear(16, 10)
        )
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 32, 28, 28]           9,248
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           9,248
            Conv2d-5           [-1, 32, 14, 14]           9,248
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 32, 5, 5]           9,248
            Conv2d-8             [-1, 32, 3, 3]           9,248
            Conv2d-9             [-1, 10, 1, 1]           2,890
Total params: 49,450
Trainable params: 49,450
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.55
Params size (MB): 0.19
Estimated Total Size (MB): 0.74
---------------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


#New parameter counts

In [5]:
import math
-math.log(0.61,2.7) - math.log(0.8,2.7) - math.log(0.5,2.7)

1.4201706878547207

In [6]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = NewNetwork().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
         Dropout2d-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           2,320
              ReLU-6           [-1, 16, 28, 28]               0
       BatchNorm2d-7           [-1, 16, 28, 28]              32
         Dropout2d-8           [-1, 16, 28, 28]               0
            Conv2d-9           [-1, 16, 28, 28]           2,320
             ReLU-10           [-1, 16, 28, 28]               0
      BatchNorm2d-11           [-1, 16, 28, 28]              32
        Dropout2d-12           [-1, 16, 28, 28]               0
        MaxPool2d-13           [-1, 16, 14, 14]               0
           Conv2d-14           [-1, 16,

In [7]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data1', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data1', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [8]:
import numpy as np
import matplotlib.pyplot as plt
#batch = next(iter(train_loader))



In [9]:
#images, labels = batch
#grid = torchvision.utils.make_grid(images, nrow=10)
#plt.figure(figsize=(15,15))
#plt.imshow(np.transpose(grid, (1,2,0)))
#print('labels:', labels)

In [10]:
from tqdm.notebook import tqdm
def train(model, device, train_loader, optimizer, epoch):
    print('In epoch {}'.format(epoch))
    model.train()
    pbar = tqdm(train_loader)
    train_loss = 0
    train_correct = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred_train = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        train_correct += pred_train.eq(target.view_as(pred_train)).sum().item()
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        train_loss, train_correct, len(train_loader.dataset),
       100. * train_correct / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:

model = NewNetwork().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#train(model, device, train_loader, optimizer, 1)
#model.train()
#pbar = tqdm(train_loader)
#enumerate(pbar)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

In epoch 1




Train set: Average loss: 5457.8876, Accuracy: 58448/60000 (97.4%)


Test set: Average loss: 0.0560, Accuracy: 9834/10000 (98.3%)

In epoch 2




Train set: Average loss: 3679.2076, Accuracy: 58925/60000 (98.2%)


Test set: Average loss: 0.0379, Accuracy: 9880/10000 (98.8%)

In epoch 3




Train set: Average loss: 3102.8645, Accuracy: 59058/60000 (98.4%)


Test set: Average loss: 0.0341, Accuracy: 9888/10000 (98.9%)

In epoch 4




Train set: Average loss: 2465.3133, Accuracy: 59282/60000 (98.8%)


Test set: Average loss: 0.0286, Accuracy: 9910/10000 (99.1%)

In epoch 5


...



Train set: Average loss: 1116.7202, Accuracy: 59632/60000 (99.4%)


Test set: Average loss: 0.0196, Accuracy: 9933/10000 (99.3%)

In epoch 16




Train set: Average loss: 987.1960, Accuracy: 59689/60000 (99.5%)


Test set: Average loss: 0.0197, Accuracy: 9934/10000 (99.3%)

In epoch 17




Train set: Average loss: 990.1321, Accuracy: 59667/60000 (99.4%)


Test set: Average loss: 0.0206, Accuracy: 9931/10000 (99.3%)

In epoch 18




Train set: Average loss: 1030.2964, Accuracy: 59675/60000 (99.5%)


Test set: Average loss: 0.0195, Accuracy: 9930/10000 (99.3%)

In epoch 19




Train set: Average loss: 832.7743, Accuracy: 59731/60000 (99.6%)


Test set: Average loss: 0.0162, Accuracy: 9945/10000 (99.5%)

